In [1]:
from jupyter_dash import JupyterDash

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import dash_bootstrap_components as dbc
import dash_html_components as html
from datetime import datetime as dt

In [85]:
# Since we're adding callbacks to elements that don't exist in the app.layout,
# Dash will raise an exception to warn us that we might be
# doing something wrong.
# In this case, we're adding the elements through a callback, so we can ignore
# the exception.

app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

df = pd.read_json('C:/Users/Utilisateur/Desktop/Cours/M1/projet inter-promo/groupe 7/donnees/df_final.json')

df = df.dropna(subset=['art_title', 'art_url','art_content','src_img']).drop(['art_id'], axis=1).reset_index(drop=True)



# Search Bar

search_bar = dbc.Row(
    [
        dbc.Col(dbc.Input(type="search", placeholder="Search")),
        dbc.Col(
            dbc.Button("Search", color="primary", className="ml-2"),
            width="auto",
        ),
    ],
    no_gutters=True,
    className="ml-auto flex-nowrap mt-3 mt-md-0",
    align="center",
)

# Nav Bar

navbar = dbc.Navbar(
    [
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    dbc.Col(dbc.NavbarBrand("Berger-levrault", className="ml-2")),
                ],
                align="center",
                no_gutters=True,
            ),
            href="https://www.berger-levrault.com/fr/",
        ),
        dbc.NavbarToggler(id="navbar-toggler"),
        dbc.Collapse(search_bar, id="navbar-collapse", navbar=True),
    ],
    color="dark",
    dark=True,
)


# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "margin-left": "2rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "2rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
    "width":"auto",
    "background-color": "#f8f9fa",
}

#Side bar

sidebar = html.Div(
    [
        html.H2("Home"),
        html.Hr(),
        html.Div([
        dbc.Col(
        [
            # -------TOUS LES DROPDOWNS + DATE PICKER ---------
            # gerer fonction du boutton voir components dash (tuto)
            dbc.Col(html.Div([html.Label('Country'),
                              dcc.Dropdown(id='comp_filt',
                    options=[
                        {'label': 'France', 'value': 'F'},
                        {'label': 'International', 'value': 'IN'}
                    ],
                    multi=True
                    )])),
            dbc.Col(html.Div([html.Label('Categorie'),
                              dcc.Dropdown(id='cat_filt',
                    options=[
                        {'label': 'Economic', 'value': 'EC'},
                        {'label': 'Ecology', 'value': 'E'}
                    ],
                    value = 'Economic',                   
                    multi=True
                    )])),
            dbc.Col(html.Div([html.Label('Date Range'), dcc.DatePickerRange(
                    id='date-picker-range',
                    start_date=dt(2016, 1, 15),
                    end_date=dt(2021, 1, 4)
                    # end_date_placeholder_text='Select a date!'
                    )])),
            dbc.Col(html.Div([html.Label('Number of article par page'), dcc.Input(
                    id = 'number',
                    type= "number",
                    value = 10,
                    min = 1
                    )])),
            html.Hr(),
            dbc.Col(html.Div([html.Label('Filters'),
                              html.Br(),
                              html.Button('Refresh Filters',
                                          id='reset_button', style={"vertical-align": "middle","border-radius": "8px",
                                                                   "background-color": "white",
                                                                    "color": "black",
                                                                    "border": "2px solid #e7e7e7"})]))
            ]),
    
        ]),
        
    ],
    style=SIDEBAR_STYLE,
)


#Nav bar for pagging

nb_pages = 10

pages = dbc.NavbarSimple(
    children=[
        dbc.Col(html.Div(html.H5("Page", style = {"color":"white"}))),
        dbc.Col(html.Div([dcc.Input(
                    id = 'num_page',
                    type= "number",
                    value = 1,
                    min = 1,
                    style = {"border-radius": "8px", "color": "black", "border": "2px solid #e7e7e7",
                            "width" : "30%",
                            "background-color": "#eee"}
                    )])),
    ],
    color="dark",
)


# Content page

card = html.Div(id ='out_card')

article = html.Div("Callback", id ='out_article')
    
content = html.Div(card,id="page-content", style=CONTENT_STYLE)

app.layout =html.Div(
    [
        navbar,
        html.Hr(),
        dbc.Row(
            [
                dbc.Col([dcc.Location(id="url"),sidebar], width = "100%"),
                dbc.Col(content),
            ],
            no_gutters=True,
        ),
        pages,
    ]
)

# Callback

# add callback for toggling the collapse on small screens
@app.callback(
    dash.dependencies.Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)

def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

# Content cards

@app.callback(
    dash.dependencies.Output('out_card', 'children'),
    [Input('number', 'value')],
    [Input('num_page','value')]
)

def update_card_div(number,num_page):
    
    
    filtered_df = df[((num_page*number)-number):((num_page*number))]
    
    cards = []
    
    for i in range(((num_page*number)-number),((num_page*number))):
        cards.append(html.Div([
            dbc.Col(html.Div([
                dbc.Card([dbc.CardImg(src=filtered_df['src_img'][i],alt="Can't display this image",top=True),
                    dbc.CardBody(
                        [
                            dbc.CardLink(html.H5(filtered_df['art_title'][i], className="card-title"), href=filtered_df['art_url'][i]),
                            html.P(filtered_df['art_content'][i][:200]),
                            dbc.Button("Learn more",id={'type': 'article-button','index': i},className="mb-3",color="primary", href ="/page-"+str(i))
                        ]
                    )],
                    className="w-100 mb-3"
                ),
            ])
            )]))
    
    return cards

#Content article

@app.callback(
    Output("out_article", "children"),
    [dash.dependencies.Input('url', 'pathname')]
)

def toggle_article(pathname):
    if pathname == "/":
        raise PreventUpdate
    else :
        
        index = int(pathname.split("-")[1])

        df_article = df.iloc[index]

        article = html.Div([
            dbc.Col(html.Div([
                dbc.Card([
                    dbc.CardImg(src=df_article['src_img'],alt="Can't display this image",top=True),
                    dbc.CardBody(
                        [
                            dbc.CardLink(html.H5(df_article['art_title'], className="card-title"), href=df_article['art_url']),
                            html.P(df_article['art_content'])
                        ]
                    )],
                    className="w-100 mb-3"
                ),
            ])
        )])


        return article

#Content    
    
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])


def render_page_content(pathname):
    if pathname == "/":
        return content
    else:
        return article
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )




if __name__ == "__main__":
    app.run_server(debug=True)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

SystemExit: 1

In [86]:
app.run_server(port=8050,host='127.0.0.1')

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run